# --- Day 12: Garden Groups ---

Why not search for the Chief Historian near the gardener and his massive farm? There's plenty of food, so The Historians grab something to eat while they search.

You're about to settle near a complex arrangement of garden plots when some Elves ask if you can lend a hand. They'd like to set up fences around each region of garden plots, but they can't figure out how much fence they need to order or how much it will cost. They hand you a map (your puzzle input) of the garden plots.

Each garden plot grows only a single type of plant and is indicated by a single letter on your map. When multiple garden plots are growing the same type of plant and are touching (horizontally or vertically), they form a region. For example:

```
AAAA
BBCD
BBCC
EEEC
```

This 4x4 arrangement includes garden plots growing five different types of plants (labeled `A`, `B`, `C`, `D`, and `E`), each grouped into their own region.

### Calculating Region Properties

To accurately calculate the cost of the fence around a single region, you need to know:
1. **Region's Area**: The number of garden plots the region contains.
2. **Region's Perimeter**: The number of sides of garden plots in the region that do not touch another garden plot in the same region.

For the map above:
- The area of type `A`, `B`, and `C` plants are each 4.
- The type `E` plants form a region of area 3.
- The type `D` plants form a region of area 1.

Perimeters:
- Regions `A` and `C` each have a perimeter of 10.
- Regions `B` and `E` each have a perimeter of 8.
- The lone `D` plot has a perimeter of 4.

Visually, the regions' perimeters are:

```
+-+-+-+-+
|A A A A|
+-+-+-+-+     +-+
              |D|
+-+-+   +-+   +-+
|B B|   |C|
+   +   + +-+
|B B|   |C C|
+-+-+   +-+ +
          |C|
+-+-+-+   +-+
|E E E|
+-+-+-+
```

### Additional Example

A more complex arrangement:

```
OOOOO
OXOXO
OOOOO
OXOXO
OOOOO
```

This contains:
- One large region of `O` plots with area 21 and perimeter 36.
- Four smaller regions of `X` plots, each with area 1 and perimeter 4.

Prices:
- Region `O`: \(21 * 36 = 756\)
- Each `X`: \(1 * 4 = 4\)

Total price: \(756 + 4 + 4 + 4 + 4 = 772\).

### Larger Example

A larger map:

```
RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
```

#### Regions and Prices
- Region of `R`: \(12 * 18 = 216\)
- Region of `I`: \(4 * 8 = 32\)
- Region of `C`: \(14 * 28 = 392\)
- Region of `F`: \(10 * 18 = 180\)
- Region of `V`: \(13 * 20 = 260\)
- Region of `J`: \(11 * 20 = 220\)
- Region of `C`: \(1 * 4 = 4\)
- Region of `E`: \(13 * 18 = 234\)
- Region of `I`: \(14 * 22 = 308\)
- Region of `M`: \(5 * 12 = 60\)
- Region of `S`: \(3 * 8 = 24\)

Total price: \(216 + 32 + 392 + 180 + 260 + 220 + 4 + 234 + 308 + 60 + 24 = 1930\).

---

### Puzzle Objective

**What is the total price of fencing all regions on your map?**

In [1]:
from dataclasses import dataclass
import numpy as np
import cv2
from scipy.ndimage import convolve
from typing import List


@dataclass
class Region:
    """Represents a region with area and perimeter."""
    plant: str
    area: int
    perimeter: int = 0
    sides: int = 0

    def fence_price(self) -> int:
        """Calculate the 'fence price' based on area and perimeter."""
        return self.area * self.perimeter
    
    def bulk_price(self) -> int:
        return self.area * self.sides


class Garden:
    """Represents a garden and provides methods to process and analyze regions."""

    def __init__(self, garden_array: np.ndarray):
        """
        Initialize a Garden instance.

        Args:
            garden_array (numpy.ndarray): Array representation of the garden layout.
        """
        self.garden_array = garden_array
        self.regions: List[Region] = []

    @classmethod
    def from_string(cls, garden_string: str):
        """
        Create a Garden instance from a multiline string.

        Args:
            garden_string (str): Multiline string representing the garden layout.

        Returns:
            Garden: A Garden instance.
        """
        rows = [list(row) for row in garden_string.strip().split("\n")]
        garden_array = np.array(rows)
        return cls(garden_array)

    @classmethod
    def from_file(cls, file_path: str):
        """
        Create a Garden instance from a file.

        Args:
            file_path (str): Path to the file containing the garden layout.

        Returns:
            Garden: A Garden instance.
        """
        with open(file_path, "r") as file:
            garden_string = file.read()
        return cls.from_string(garden_string)

    def compute_regions_with_contours(self):
        """Process the garden layout and compute regions with area and perimeter."""
        # Convert garden characters to numeric representation
        garden_nums = np.vectorize(ord)(self.garden_array).astype(np.uint8)
        unique_plants = np.unique(garden_nums)

        # Process each plant type
        for plant in unique_plants:
            # Create a binary mask for the current plant type
            binary_mask = (garden_nums == plant).astype(np.uint8)

            # Find contours and their hierarchy
            contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
            if hierarchy is None:
                continue  # Skip if no contours found

            # Cluster contours and compute region properties
            clusters = self._cluster_contours(contours, hierarchy)
            for cluster in clusters:
                filled_mask = cv2.drawContours(np.zeros_like(binary_mask), cluster, -1, 1, cv2.FILLED)
                area = filled_mask.sum().item()
                perimeter = self._compute_perimeter_with_convolution(filled_mask)
                self.regions.append(Region(chr(plant), area, perimeter=perimeter))

    def compute_regions(self):
        """Process the garden layout and compute regions with area and perimeter."""
        # Convert garden characters to numeric representation
        garden_nums = np.vectorize(ord)(self.garden_array).astype(np.uint8)
        unique_plants = np.unique(garden_nums)

        for plant in unique_plants:
            # Create a binary mask for the current plant type
            binary_mask = (garden_nums == plant).astype(np.uint8)

            # Use connectedComponentsWithStats with 4-connectivity
            num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_mask, connectivity=4)

            # Process each labeled region
            for label in range(1, num_labels):  # Skip label 0 (background)
                area = stats[label, cv2.CC_STAT_AREA].item()
                # Generate a mask for the current region
                region_mask = (labels == label).astype(np.uint8)
                perimeter = self._compute_perimeter_with_convolution(region_mask)
                self.regions.append(Region(chr(plant), area, perimeter=perimeter))

    def total_fence_price(self) -> int:
        """Calculate the total fence price for all regions."""
        return sum(region.fence_price() for region in self.regions)

    @staticmethod
    def _compute_perimeter_with_convolution(binary_mask: np.ndarray) -> int:
        """
        Compute the exact perimeter of a grid-aligned binary shape.

        Args:
            binary_mask (numpy.ndarray): Binary mask of the shape.

        Returns:
            int: Exact perimeter of the shape.
        """
        # Define a kernel for 4-connectivity
        kernel = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]], dtype=int)

        # Convolve the binary mask with the kernel to count neighbors
        filled_neighbors = convolve(binary_mask.astype(int), kernel, mode="constant", cval=0)

        # Count exposed edges for shape pixels
        perimeter = np.sum(4 - filled_neighbors[binary_mask == 1])
        return perimeter.item()

    @staticmethod
    def _cluster_contours(contours: list, hierarchy: np.ndarray) -> list:
        """
        Group contours into clusters based on hierarchy (parent-child relationships).

        Args:
            contours (list): List of contour points.
            hierarchy (numpy.ndarray): Contour hierarchy from cv2.findContours.

        Returns:
            list: List of clusters, where each cluster is a list of contours.
        """
        clusters = {}
        for idx, (_, _, _, parent) in enumerate(hierarchy[0]):
            if parent == -1:  # Root contour
                clusters[idx] = [idx]
            else:  # Append child to parent's cluster
                clusters.setdefault(parent, []).append(idx)
        return [[contours[i] for i in indices] for indices in clusters.values()]


# Example usage
garden_string = """
OOOOO
OXOXO
OOOOO
OXOXO
OOOOO
"""

# Initialize garden from string
garden = Garden.from_string(garden_string)

# Compute regions
garden.compute_regions()

# Output results
print(f"Regions: {garden.regions}")
print(f"Total Fence Price: {garden.total_fence_price()}")


Regions: [Region(plant='O', area=21, perimeter=36, sides=0), Region(plant='X', area=1, perimeter=4, sides=0), Region(plant='X', area=1, perimeter=4, sides=0), Region(plant='X', area=1, perimeter=4, sides=0), Region(plant='X', area=1, perimeter=4, sides=0)]
Total Fence Price: 772


In [2]:
garden = Garden.from_file("./example.txt")

# Compute regions
garden.compute_regions()

# Output results
print(f"Regions: {garden.regions}")
print(f"Total Fence Price: {garden.total_fence_price()}")

Regions: [Region(plant='C', area=14, perimeter=28, sides=0), Region(plant='C', area=1, perimeter=4, sides=0), Region(plant='E', area=13, perimeter=18, sides=0), Region(plant='F', area=10, perimeter=18, sides=0), Region(plant='I', area=4, perimeter=8, sides=0), Region(plant='I', area=14, perimeter=22, sides=0), Region(plant='J', area=11, perimeter=20, sides=0), Region(plant='M', area=5, perimeter=12, sides=0), Region(plant='R', area=12, perimeter=18, sides=0), Region(plant='S', area=3, perimeter=8, sides=0), Region(plant='V', area=13, perimeter=20, sides=0)]
Total Fence Price: 1930


In [3]:
garden = Garden.from_file("./input.txt")

# findContours takes into account 8-connectivity, wrong answer!!!
garden.compute_regions_with_contours()
print(f"Total Fence Price: {garden.total_fence_price()}")

Total Fence Price: 1450946


In [4]:
garden = Garden.from_file("./input.txt")

# Compute regions
garden.compute_regions()
print(f"Total Fence Price: {garden.total_fence_price()}")

Total Fence Price: 1437300


# Part Two: Bulk Discount Fencing

Fortunately, the Elves are ordering so much fence that they qualify for a **bulk discount**! 

Under the bulk discount, instead of using the **perimeter** to calculate the price, you need to use the **number of sides** each region has. Each straight section of fence counts as a side, regardless of its length.

---

### Example 1

Consider this example again:

```
AAAA
BBCD
BBCC
EEEC
```

- The region containing type **A** plants has **4 sides**, as does each of the regions containing plants of type **B**, **D**, and **E**.
- However, the more complex region containing the plants of type **C** has **8 sides**.

Using the new method of calculating the per-region price by multiplying the region's **area** by its **number of sides**, the prices are:

- **A**: \(4 * 4 = 16\)
- **B**: \(4 * 4 = 16\)
- **C**: \(8 * 4 = 32\)
- **D**: \(4 * 1 = 4\)
- **E**: \(4 * 3 = 12\)

**Total Price**: \(16 + 16 + 32 + 4 + 12 = 80\).

---

### Example 2

For the second example (full of type **X** and **O** plants):

```
OOOOO
OXOXO
OOOOO
OXOXO
OOOOO
```

The **total price** would be **436**.

---

### Example 3: E-Shaped Region

Here’s another map that includes an E-shaped region full of type **E** plants:

```
EEEEE
EXXXX
EEEEE
EXXXX
EEEEE
```

- The **E-shaped region** has:
  - **Area**: 17
  - **Sides**: 12  
  **Price**: \(17 * 12 = 204\)

- Including the two regions full of type **X** plants, this map has a **total price of 236**.

---

### Example 4

This map has a **total price of 368**:

```
AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA
```

- It includes:
  - **Two regions full of type B plants**, each with **4 sides**.
  - A single region full of type **A plants** with:
    - **4 sides on the outside**.
    - **8 more sides on the inside** (total of 12 sides).

⚠️ **Be careful when counting the fence around regions like the one full of type A plants.** Each section of the fence has an **in-side** and an **out-side**, so the fence does not connect across the middle of the region (where the two B regions touch diagonally).  
(The Elves would have used the Möbius Fencing Company instead, but their contract terms were too one-sided!)

---

### Example 5: Larger Map

The larger example from before now has the following updated prices:

| Plant Type | Area | Sides | Price            |
|------------|------|-------|------------------|
| **R**      | 12   | 10    | \(12 * 10 = 120\) |
| **I**      | 4    | 4     | \(4 * 4 = 16\)    |
| **C**      | 14   | 22    | \(14 * 22 = 308\) |
| **F**      | 10   | 12    | \(10 * 12 = 120\) |
| **V**      | 13   | 10    | \(13 * 10 = 130\) |
| **J**      | 11   | 12    | \(11 * 12 = 132\) |
| **C**      | 1    | 4     | \(1 * 4 = 4\)     |
| **E**      | 13   | 8     | \(13 * 8 = 104\)  |
| **I**      | 14   | 16    | \(14 * 16 = 224\) |
| **M**      | 5    | 6     | \(5 * 6 = 30\)    |
| **S**      | 3    | 6     | \(3 * 6 = 18\)    |

**New Total Price**: \(120 + 16 + 308 + 120 + 130 + 132 + 4 + 104 + 224 + 30 + 18 = 1206\).

---

### Task

What is the **new total price** of fencing all regions on your map?

In [5]:
class GardenV2(Garden):
    def compute_regions_with_bulk(self):
        """Process the garden layout and compute regions with area and perimeter."""
        # Convert garden characters to numeric representation
        garden_nums = np.vectorize(ord)(self.garden_array).astype(np.uint8)
        unique_plants = np.unique(garden_nums)

        for plant in unique_plants:
            # Create a binary mask for the current plant type
            binary_mask = (garden_nums == plant).astype(np.uint8)

            # Use connectedComponentsWithStats with 4-connectivity
            num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(
                binary_mask, connectivity=4
            )

            # Process each labeled region
            for label in range(1, num_labels):  # Skip label 0 (background)
                area = stats[label, cv2.CC_STAT_AREA].item()
                # Generate a mask for the current region
                region_mask = (labels == label).astype(np.uint8)
                n_lines = self.count_lines(region_mask)
                self.regions.append(Region(chr(plant), area, sides=n_lines))

    @staticmethod
    def count_lines(binary_mask: np.ndarray):
        """
        Count the number of straight line segments in a contour using NumPy operations.

        Args:
            contour (numpy.ndarray): The contour points as a Nx1x2 array.

        Returns:
            int: Number of straight line segments.
        """
        binary_mask = cv2.resize(binary_mask, None, fx=2, fy=2, interpolation=cv2.INTER_NEAREST)
        contours, _ = cv2.findContours(
            binary_mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE
        )
        total_lines = 0
        for contour in contours:
            # Extract the points as a 2D array of shape (N, 2)
            points = contour[:, 0]

            # Compare consecutive points (cyclically, including last -> first)
            diff = points - np.roll(points, shift=-1, axis=0)

            # Identify horizontal and vertical segments
            is_horizontal_or_vertical = (diff[:, 0] == 0) | (diff[:, 1] == 0)

            # Count the valid segments
            total_lines += np.sum(is_horizontal_or_vertical).item()
        return total_lines
    
    def total_bulk_price(self):
        """Calculate the total bulk price for all regions."""
        return sum(region.bulk_price() for region in self.regions) 
    


garden_string = """
EEEEE
EXXXX
EEEEE
EXXXX
EEEEE
"""

# Initialize garden from string
garden = GardenV2.from_string(garden_string)

# Compute regions
garden.compute_regions_with_bulk()

# Output results
print(f"Regions: {garden.regions}")
print(f"Total Fence Price: {garden.total_bulk_price()}")

Regions: [Region(plant='E', area=17, perimeter=0, sides=12), Region(plant='X', area=4, perimeter=0, sides=4), Region(plant='X', area=4, perimeter=0, sides=4)]
Total Fence Price: 236


In [6]:
garden_string = """
AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA
"""

# Initialize garden from string
garden = GardenV2.from_string(garden_string)

# Compute regions
garden.compute_regions_with_bulk()

# Output results
print(f"Regions: {garden.regions}")
print(f"Total Fence Price: {garden.total_bulk_price()}")

Regions: [Region(plant='A', area=28, perimeter=0, sides=12), Region(plant='B', area=4, perimeter=0, sides=4), Region(plant='B', area=4, perimeter=0, sides=4)]
Total Fence Price: 368


In [7]:
# Initialize garden from string
garden = GardenV2.from_file("./example.txt")

# Compute regions
garden.compute_regions_with_bulk()
print(f"Total Fence Price: {garden.total_bulk_price()}")

Total Fence Price: 1206


In [8]:
# Initialize garden from string
garden = GardenV2.from_file("./input.txt")

# Compute regions
garden.compute_regions_with_bulk()
print(f"Total Fence Price: {garden.total_bulk_price()}")

Total Fence Price: 849332
